In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch transformers sentence-transformers nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # Explicitly download the missing resource
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import torch

In [ ]:
# Load BART model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-finetuned-squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/bart-large-finetuned-squadv1")

#tokenizer = AutoTokenizer.from_pretrained("Praise2112/ModernBERT-large-squad2-v0.1", trust_remote_code=True)
#model = AutoModelForQuestionAnswering.from_pretrained("Praise2112/ModernBERT-large-squad2-v0.1", trust_remote_code=True)


pipe = pipeline("text2text-generation", model='lmqg/t5-large-squad-qg')

# Load Sentence-BERT model
sentence_bert_model = SentenceTransformer('average_word_embeddings_glove.6B.300d')

# Define the event handler for the button click
def generate_flashcards(text, num_flashcards):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Generate questions
    questions = []
    for sentence in sentences:
        input_text = 'generate question: ' + sentence
        question = pipe(input_text)
        question = question[0]['generated_text']
        questions.append(question)

    # Rank questions based on quality/relevance
    question_embeddings = sentence_bert_model.encode(questions)
    ranking_scores = question_embeddings.dot(question_embeddings.T).mean(axis=1)  # Compute cosine similarity scores

    # Filter and select the most significant questions
    selected_indices = ranking_scores.argsort()[-num_flashcards:][::-1]
    selected_questions = [questions[i] for i in selected_indices]

    answers = []
    for question in selected_questions:
        inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"].tolist()[0]

        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

        outputs = model(**inputs)
        answer_start_scores=outputs.start_logits
        answer_end_scores=outputs.end_logits

        answer_start = torch.argmax(
            answer_start_scores
        )  # Get the most likely beginning of answer with the argmax of the score
        answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        # Combine the tokens in the answer and print it out.""
        answer = answer.replace("#","")

        answers.append(answer)


    flashcards = []
    for i in range(len(selected_questions)):
      flashcard = {
          'question': selected_questions[i],
          'answer': answers[i]
      }
      flashcards.append(flashcard)

    return flashcards

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

Device set to use cuda:0


KeyboardInterrupt: 

Testing stuff

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

# Load BART model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-finetuned-squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/bart-large-finetuned-squadv1")
pipe = pipeline("text2text-generation", model='lmqg/t5-large-squad-qg')

# Load Sentence-BERT model
sentence_bert_model = SentenceTransformer('average_word_embeddings_glove.6B.300d')

def generate_flashcards(text, num_flashcards):
    sentences = sent_tokenize(text)

    # Group sentences into paragraphs (sliding window approach)
    window_size = 3  # Number of sentences in a chunk
    sentence_chunks = [" ".join(sentences[i:i+window_size]) for i in range(0, len(sentences), window_size)]

    # Generate questions
    questions = []
    for chunk in sentence_chunks:
        input_text = 'generate question: ' + chunk
        question = pipe(input_text)[0]['generated_text']
        questions.append((question, chunk))  # Store question + context

    # Rank questions based on relevance
    question_texts = [q[0] for q in questions]
    question_embeddings = sentence_bert_model.encode(question_texts)
    ranking_scores = np.mean(question_embeddings.dot(question_embeddings.T), axis=1)

    # Select the most significant questions
    selected_indices = ranking_scores.argsort()[-num_flashcards:][::-1]
    selected_questions = [questions[i] for i in selected_indices]

    # Generate answers using the best context
    answers = []
    for question, best_context in selected_questions:
        inputs = tokenizer.encode_plus(question, best_context, add_special_tokens=True, return_tensors="pt")
        outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"].tolist()[0][answer_start:answer_end]))
        answers.append(answer.replace("#", ""))

    # Construct flashcards
    flashcards = [{"question": q, "answer": a} for (q, _), a in zip(selected_questions, answers)]
    return flashcards


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

whitespacetokenizer_config.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/480M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
text = """AI models are essentially algorithms trained on vast amounts of data.
They learn patterns and relationships within that data, enabling them to perform tasks like generating text, recognizing images, or predicting outcomes. Different types of models exist, each suited for specific purposes.
Some focus on language processing, others on image analysis, and some on complex data analysis.
The accuracy and effectiveness of an AI model depend heavily on the quality and quantity of the data it's trained on.
Ongoing research aims to improve model efficiency, reduce bias, and enhance their ability to generalize to new situations."""

generate_flashcards(text, 5)

[{'question': 'What are some of the tasks that AI models can perform?',
  'answer': ' generating text, recognizing images, or predicting outcomes'},
 {'question': 'What is the focus of some of the AI models?',
  'answer': ' language processing'}]

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

# Set your Ngrok auth token
ngrok.set_auth_token("YOUR_NGROK_TOKEN")


In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import re  # For sentence splitting
from typing import List

# Allow Colab to run FastAPI event loop
nest_asyncio.apply()

# Define FastAPI app
app = FastAPI()

class FlashcardRequest(BaseModel):
    text: str
    num_flashcards: int

@app.post("/generate_flashcards")
async def generate_flashcards_endpoint(request: FlashcardRequest):
    flashcards = generate_flashcards(request.text, request.num_flashcards)
    return {"flashcards": flashcards, "num_flashcards": request.num_flashcards}

# Start Ngrok tunnel
public_url = ngrok.connect(8000).public_url
print(f"Ngrok Tunnel URL: {public_url}")

# Start FastAPI server in Colab
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [1520]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Ngrok Tunnel URL: https://36f3-34-87-105-189.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1520]
